In [ ]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
from config import app_token
from config import username
from config import password

#dependencies for 2nd data source
import pandas as pd
import scipy.stats as st
import numpy as np

In [ ]:
# Getting the data from API NY Open Data
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)

In [ ]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"


# Read the data into df
cws_df = pd.read_csv(data_path)


#Preview data
cws_df.head()